In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
import pandas as pd
import numpy as np
import time
import datetime
import sys
#need attention
import multiprocessing.dummy as mp
from lxml import etree
import requests
import logging
import pickle
import json
import string
from zhon.hanzi import punctuation
from threading import Thread
from functools import wraps
import re
import warnings
import ddddocr
import random
import argparse
from  PIL import Image
import shutil
warnings.filterwarnings("ignore")

/Users/qianyuyang/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
start_url = "https://atlas.eia.gov/search"
chromedriver = "/Users/qianyuyang/Desktop/Qianyu/RA/energy_database/driver/chromedriver"
options = webdriver.ChromeOptions()
prefs = {"download.default_directory":"/Users/qianyuyang/Downloads/%s"%1}
    

# 谷歌文档提到需要加上这个属性来规避bug
options.add_argument('--disable-gpu') 
# 不加载图片, 提升速度
#options.add_argument('blink-settings=imagesEnabled=false') 
# 浏览器不提供可视化页面. linux下如果系统不支持可视化不加这条会启动失败
options.add_argument('--headless')
#options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(executable_path = chromedriver,options = options)
driver.get(start_url)
#num_flag = True
driver.maximize_window()



In [3]:
xpath = {
    'dataset_button':'//div[@class="col-sm-12"]/h3/div/a',
    'load_button':'//button[@class="btn more-results link-color-primary"]',
    'level_2_button':'//div[@class="stacked-icon"]',
    'detail_button':'//div/a[contains(@class,"btn")]',
    'intro_cordinate':'//div[@class="content-summary"]/div/div/div/div',
    'download_panel_header':'//h1[@class="download-panel-header"]'
}


In [4]:
while True:
    try:
        WebDriverWait(driver,10,1).until(EC.presence_of_element_located((By.XPATH,xpath['load_button']))).click()
        time.sleep(3)
    except:
        break
        

In [5]:
html_source = driver.page_source
label = etree.HTML(html_source)
all_dataset_name = label.xpath('//div[@class="col-sm-12"]/h3/div/a/text()')

In [6]:
all_dataset_name = [i.replace("\n","") for i in all_dataset_name]
all_dataset_name = [" ".join([j for j in i.split(" ") if j != '']) for i in all_dataset_name]
all_dataset_name

['USA Urban Areas (below 1:500k)',
 'USA Urban Areas (1:500k-1.5M)',
 'USA Urban Areas (over 1:1.5M)',
 'USA Census Populated Place Areas',
 'USA Urban Areas',
 'NOAA Tornado Warnings',
 'NOAA Severe Thunderstorm Warnings',
 'NOAA Flash Flood Warnings',
 'NOAA Special Marine Warnings',
 'MODIS Thermal (Last 48 hours)',
 'MODIS Thermal (Last 7 days)',
 'Observed Wind Swath',
 'Strong Tropical Storm (50kts)',
 'Raw 1/10th Degree Data (All)',
 'Forecast Error Cone',
 'Tropical Storm Force (34kts)',
 'Observed Wind Swath',
 'Hurricane Force (64kts+)',
 'Watches and Warnings',
 'Satellite (VIIRS) Thermal Hotspots and Fire Activity',
 'National Weather Service Wind Speed Forecast',
 'National Weather Service Smoke Forecast',
 'Accumulation by Time',
 'Amount by Time',
 'USA 117th Congressional Districts',
 'USA Counties (Generalized)',
 'Public Forecast Zones',
 'Fire Forecast Zones',
 'Coastal and Offshore Marine Zones',
 'Events Ordered by Size and Severity',
 'US States and Territories',


In [11]:
prev = []
yb = []
for i in all_dataset_name:
    if i in prev:
        print(i)
        yb.append(i + "*")
    else:
        yb.append(i)
    prev.append(i)
    

Observed Wind Swath
Observed Position
Observed Track


In [11]:
def get_latest_download_file(file_dir="/Users/qianyuyang/Downloads/0"):
    
    l=os.listdir(file_dir)
    l.sort(key=lambda fn: os.path.getmtime(file_dir+"/"+fn) if not os.path.isdir(file_dir+"/"+fn) else 0)

    #get file dowload time
    d=datetime.datetime.fromtimestamp(os.path.getmtime(file_dir+"/"+l[-1]))
    print(d)
    print(l)
    return l[-1]


In [12]:
os.listdir("/Users/qianyuyang/Downloads/1")

['.DS_Store',
 'USA_Weather_Watches_and_Warnings.kml',
 'National_Weather_Service_Precipitation_Forecast.kml.crdownload',
 'Satellite_(VIIRS)_Thermal_Hotspots_and_Fire_Activity.kml',
 'Satellite_(VIIRS)_Thermal_Hotspots_and_Fire_Activity.csv']

In [13]:
get_latest_download_file("/Users/qianyuyang/Downloads/1")

2023-03-15 08:40:57.111175
['.DS_Store', 'Satellite_(VIIRS)_Thermal_Hotspots_and_Fire_Activity.csv', 'Satellite_(VIIRS)_Thermal_Hotspots_and_Fire_Activity.kml', 'National_Weather_Service_Precipitation_Forecast.kml.crdownload', 'USA_Weather_Watches_and_Warnings.kml']


'USA_Weather_Watches_and_Warnings.kml'

In [7]:
data_dict = dict()
for a,b in enumerate(all_dataset_name):
    data_dict[b] = a
data_dict

{'USA Urban Areas (below 1:500k)': 0,
 'USA Urban Areas (1:500k-1.5M)': 1,
 'USA Urban Areas (over 1:1.5M)': 2,
 'USA Census Populated Place Areas': 3,
 'USA Urban Areas': 4,
 'NOAA Tornado Warnings': 5,
 'NOAA Severe Thunderstorm Warnings': 6,
 'NOAA Flash Flood Warnings': 7,
 'NOAA Special Marine Warnings': 8,
 'MODIS Thermal (Last 48 hours)': 9,
 'MODIS Thermal (Last 7 days)': 10,
 'Observed Wind Swath': 16,
 'Strong Tropical Storm (50kts)': 12,
 'Raw 1/10th Degree Data (All)': 13,
 'Forecast Error Cone': 14,
 'Tropical Storm Force (34kts)': 15,
 'Hurricane Force (64kts+)': 17,
 'Watches and Warnings': 18,
 'Satellite (VIIRS) Thermal Hotspots and Fire Activity': 19,
 'National Weather Service Wind Speed Forecast': 20,
 'National Weather Service Smoke Forecast': 21,
 'Accumulation by Time': 22,
 'Amount by Time': 23,
 'USA 117th Congressional Districts': 24,
 'USA Counties (Generalized)': 25,
 'Public Forecast Zones': 26,
 'Fire Forecast Zones': 27,
 'Coastal and Offshore Marine Zone

In [19]:
driver.get('chrome://settings/')
driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.6);')

start_url = "https://atlas.eia.gov/search"
chromedriver = "/Users/qianyuyang/Desktop/Qianyu/RA/energy_database/driver/chromedriver"
options = webdriver.ChromeOptions()

# document in google mention this trick to avoid bug
options.add_argument('--disable-gpu') 
# not load picture
#options.add_argument('blink-settings=imagesEnabled=false') 
# do not use the visable broswer
# self.options.add_argument('--headless')

#driver = webdriver.Chrome(executable_path = chromedriver,options = options)
driver.get(start_url)
#num_flag = True
driver.maximize_window()

rest_data = [v for k,v in data_dict.items() if k not in os.listdir('../data')]
for i in rest_data:
    driver.get(start_url)
    while len(driver.find_elements_by_xpath(xpath['dataset_button'])) <= i:
        try:
            WebDriverWait(driver,10,1).until(EC.presence_of_element_located((By.XPATH,xpath['load_button']))).click()
            time.sleep(5)
        except:
            break
    WebDriverWait(driver,30,1).until(EC.presence_of_all_elements_located((By.XPATH,xpath['dataset_button'])))
    current_latest_download_file = get_latest_download_file()
    current_dataset = all_dataset_name[i]
    if not os.path.exists("../data/%s"%current_dataset):
        os.mkdir("../data/%s"%current_dataset)
    button = driver.find_elements_by_xpath(xpath['dataset_button'])[i]
    driver.execute_script("arguments[0].click();",button)
    time.sleep(random.randint(20,30))
    try:
        WebDriverWait(driver,60,1).until(EC.presence_of_all_elements_located((By.XPATH,xpath['level_2_button'])))
        
        WebDriverWait(driver,60,1).until(EC.presence_of_all_elements_located((By.XPATH,xpath['detail_button'])))
    
    except:
        continue
    time.sleep(random.randint(5,10))
    button = driver.find_element_by_xpath(xpath['detail_button'])
    driver.execute_script("arguments[0].click();",button)
    time.sleep(random.randint(10,20))
    html_source = driver.page_source
    label = etree.HTML(html_source)
    intro_text_list = label.xpath(xpath['intro_cordinate'])
    summary = [itl.xpath("string(.)") for itl in intro_text_list]
    summary = [" ".join([jj for jj in s.split(" ") if jj != '']) for s in summary]
    summary = "\n".join(summary)
    with open("../data/%s/summary.txt"%current_dataset,'w') as f:
        f.write(summary)
        f.close()
    var = 0
    var_dict = {}
    while True:
        print(var_dict)
        try:
            varibale_container = '''return document.querySelector('arcgis-hub-attributes-list').shadowRoot.querySelectorAll('calcite-accordion-item')[%s].shadowRoot.querySelector('div[class="header-text"]')'''%var
            sd = driver.execute_script(varibale_container)
            var_name = sd.find_elements_by_tag_name('span')[0].text
            var_dict[var_name] = ""
            var += 1
            try:
                description = sd.find_elements_by_tag_name('span')[1].text
                var_dict[var_name] = description
            except:
                continue
        except:
            break
    with open("../data/%s/description.json"%current_dataset,'w') as outfile:
        json.dump(var_dict, outfile)
            
    WebDriverWait(driver,60,1).until(EC.presence_of_all_elements_located((By.XPATH,xpath['detail_button'])))
    time.sleep(random.randint(10,15))
    
    button = driver.find_elements_by_xpath(xpath['level_2_button'])[1]
    driver.execute_script("arguments[0].click();",button)
    time.sleep(random.randint(20,30))
    
    
    file_num = driver.find_elements_by_xpath('//div[@class="dataset-download-card"]')
    for di in range(len(file_num)):
        try:
            download_button = '''return document.querySelectorAll('div[class="dataset-download-card"] hub-download-card')[%s].shadowRoot.querySelector('calcite-button')'''%di

            driver.execute_script(download_button).click()
            
            time.sleep(3)
            download_button = '''return document.querySelectorAll('div[class="dataset-download-card"] hub-download-card')[%s].shadowRoot.querySelectorAll('calcite-dropdown-item')[1]'''%di

            driver.execute_script(download_button).click()
        except AttributeError:
            time.sleep(15)
            while get_latest_download_file() == current_latest_download_file or 'google' in get_latest_download_file() or 'download' in get_latest_download_file():
                time.sleep(5)
            
            current_latest_download_file = get_latest_download_file()
            
            shutil.move(os.path.join("/Users/qianyuyang/Downloads",current_latest_download_file),"../data/%s"%current_dataset)
        
            time.sleep(random.randint(10,15))
            continue
    
        
        time.sleep(15)
        while get_latest_download_file() == current_latest_download_file or 'google' in get_latest_download_file() or 'download' in get_latest_download_file():
            time.sleep(5)
        
        current_latest_download_file = get_latest_download_file()
        
        shutil.move(os.path.join("/Users/qianyuyang/Downloads",current_latest_download_file),"../data/%s"%current_dataset)
    
        time.sleep(random.randint(10,15))
    
    
driver.quit()

    

{}
{'WFO': ''}
{'WFO': '', 'ETN': ''}
{'WFO': '', 'ETN': '', 'UGC': ''}
{'WFO': '', 'ETN': '', 'UGC': '', 'WFILE': ''}
{'WFO': '', 'ETN': '', 'UGC': '', 'WFILE': '', 'UTC_EXPIRE': ''}
{'WFO': '', 'ETN': '', 'UGC': '', 'WFILE': '', 'UTC_EXPIRE': '', 'UTC_ISSUE': ''}
{'WFO': '', 'ETN': '', 'UGC': '', 'WFILE': '', 'UTC_EXPIRE': '', 'UTC_ISSUE': '', 'Shape__Area': ''}
{'WFO': '', 'ETN': '', 'UGC': '', 'WFILE': '', 'UTC_EXPIRE': '', 'UTC_ISSUE': '', 'Shape__Area': '', 'Shape__Length': ''}


In [15]:
download_button = '''return document.querySelectorAll('div[class="dataset-download-card"] hub-download-card')[%s].shadowRoot.querySelectorAll('calcite-dropdown-item')[1]'''%di

driver.execute_script(download_button).click()

In [10]:
download_button = '''return document.querySelector('arcgis-hub-attributes-list').shadowRoot.querySelectorAll('calcite-accordion-item')[1].shadowRoot.querySelector('div[class="header-text"]')'''
sd = driver.execute_script(download_button)


In [12]:
sd.find_elements_by_tag_name('span')[1].text

IndexError: list index out of range

In [65]:
driver.get('chrome://settings/')
driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.6);')

In [110]:
download_button = '''return document.querySelectorAll('div[class="dataset-download-card"] hub-download-card')[%s].shadowRoot.querySelector('calcite-button')'''%3

driver.execute_script(download_button).click()

download_button = '''return document.querySelectorAll('div[class="dataset-download-card"] hub-download-card')[%s].shadowRoot.querySelectorAll('calcite-dropdown-item')[1]'''%3

driver.execute_script(download_button).click()

AttributeError: 'NoneType' object has no attribute 'click'

In [15]:
shutil.copy("/Users/qianyuyang/Desktop/Qianyu/RA/energy_database/data/11/data/USA Urban Areas (below 1:500k)/summary.txt","/Users/qianyuyang/Desktop/Qianyu/RA/energy_database/data/11/code")

'/Users/qianyuyang/Desktop/Qianyu/RA/energy_database/data/11/code/summary.txt'

In [21]:
#driver = webdriver.Chrome(executable_path = chromedriver,options = options)
driver.get(start_url)
#num_flag = True
driver.maximize_window()

In [48]:
button = driver.find_element_by_xpath(xpath['detail_button'])
driver.execute_script("arguments[0].click();",button)

In [50]:
html_source = driver.page_source
label = etree.HTML(html_source)
intro_text_list = label.xpath(xpath['intro_cordinate'])
[i.xpath("string(.)") for i in intro_text_list]

['\n      \n        This layer presents the 2010 Census Urbanized Areas (UA) and Urban Clusters (UC) at small scales and the urban areas of the Digital Chart of the World at large scales.\n      This layer presents the Census 2010 Urbanized Areas (UA) and Urban Clusters (UC). A UA consists of contiguous, densely settled census block groups (BGs) and census blocks that meet minimum population density requirements (1000ppsm /500ppsm), along with adjacent densely settled census blocks that together encompass a population of at least 50,000 people. A UC consists of contiguous, densely settled census BGs and census blocks that meet minimum population density requirements, along with adjacent densely settled census blocks that together encompass a population of at least 2,500 people, but fewer than 50,000 people. The dataset covers the 50 States plus the District of Columbia within United States.\n    \n    ']